# Calculate emissions changes in well-mixed gas levels
Calculate the emissions changes in well-mixed gases at hemisphereic and globally averaged levels. 

In [ ]:
import netCDF4 as nc
import pandas as pd
import numpy as np

In [ ]:
# Input gas levels
input_folder = "../input/"
input_co2_mole = "mole-fraction-of-carbon-dioxide-in-air_input4MIPs_GHGConcentrations_ScenarioMIP_UoM-MESSAGE-GLOBIOM-ssp245-1-2-0_gr1-GMNHSH_2015-2500.nc"
input_no2_mole = "mole-fraction-of-nitrous-oxide-in-air_input4MIPs_GHGConcentrations_ScenarioMIP_UoM-MESSAGE-GLOBIOM-ssp245-1-2-1_gr1-GMNHSH_201501-250012.nc"
input_ch4_mole = "mole-fraction-of-methane-in-air_input4MIPs_GHGConcentrations_ScenarioMIP_UoM-MESSAGE-GLOBIOM-ssp245-1-2-0_gr1-GMNHSH_201501-250012.nc"

input_co2_air = "CO2-em-AIR-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"

# Modification factors
mod_input_folder = "../../COVID19_emissions_data/global_pathways/"
mod_input_file = "Base_pathway.xlsx"

files_to_global = []

In [ ]:
for fileind in range(len(files_to_global)):
    file = files_to_blip[fileind]
    data = copy_netcdf_file(file)
    output = data.variables[key_variables[fileind]][:, :, :, :]
    for time in global_times:
        timeind = np.where(data.variables["time"][:] == time)[0]
         output[timeind, :, :, :] *= time_factor(time)